In [203]:
import pandas as pd
import json
from shapely.geometry import Point, shape

In [204]:
edges=pd.read_csv('./cities/Detroit/clean/comb_network_sim_edges.csv')
nodes=pd.read_csv('./cities/Detroit/clean/comb_network_sim_nodes.csv')

In [205]:
FLOYD_PREDECESSOR_PATH='./cities/Detroit/clean/fw_result_pt.json'
INT_NET_DF_FLOYD_PATH='./cities/Detroit/clean/sim_net_df_floyd_pt.csv'
INT_NET_COORDINATES_PATH='./cities/Detroit/clean/sim_net_node_coords_pt.json'
SIM_AREA_PATH='./cities/Detroit/clean/table_area.geojson'

In [206]:
SIM_AREA_PATH='./cities/Detroit/clean/table_area.geojson'

sim_area=json.load(open(SIM_AREA_PATH))

full_area=[shape(f['geometry']) for f in sim_area['features']]
bounds=[shp.bounds for shp in full_area]
boundsAll=[min([b[0] for b in bounds]), #W
               min([b[1] for b in bounds]), #S
               max([b[2] for b in bounds]), #E
               max([b[3] for b in bounds])] #N

In [207]:
edges.head()

,Unnamed: 0,access,area,bridge,distance,from,hgv,highway,edge_id,lanes,...,service,to,toll,tunnel,unique_agency_id,unique_route_id,unique_trip_id,weight,from_int,to_int
0,0,NaN,NaN,NaN,NaN,2010_detroit_department_of_transportation,NaN,NaN,155809020_detroit_department_of_transportation_1,NaN,...,NaN,3545_detroit_department_of_transportation,NaN,NaN,detroit_department_of_transportation,3_detroit_department_of_transportation,155809020_detroit_department_of_transportation,0.416667,1670,3017
1,1,NaN,NaN,NaN,NaN,3545_detroit_department_of_transportation,NaN,NaN,155809020_detroit_department_of_transportation_2,NaN,...,NaN,3547_detroit_department_of_transportation,NaN,NaN,detroit_department_of_transportation,3_detroit_department_of_transportation,155809020_detroit_department_of_transportation,0.750000,3017,3019
2,2,NaN,NaN,NaN,NaN,3547_detroit_department_of_transportation,NaN,NaN,155809020_detroit_department_of_transportation_3,NaN,...,NaN,3551_detroit_department_of_transportation,NaN,NaN,detroit_department_of_transportation,3_detroit_department_of_transportation,155809020_detroit_department_of_transportation,0.800000,3019,3023
3,3,NaN,NaN,NaN,NaN,3551_detroit_department_of_transportation,NaN,NaN,155809020_detroit_department_of_transportation_4,NaN,...,NaN,3548_detroit_department_of_transportation,NaN,NaN,detroit_department_of_transportation,3_detroit_department_of_transportation,155809020_detroit_department_of_transportation,0.700000,3023,3020
4,4,NaN,NaN,NaN,NaN,3548_detroit_department_of_transportation,NaN,NaN,155809020_detroit_department_of_transportation_5,NaN,...,NaN,3552_detroit_department_of_transportation,NaN,NaN,detroit_department_of_transportation,3_detroit_department_of_transportation,155809020_detroit_department_of_transportation,0.800000,3020,3024


In [208]:
edges.columns

Index(['Unnamed: 0', 'access', 'area', 'bridge', 'distance', 'from', 'hgv',
       'highway', 'edge_id', 'lanes', 'maxspeed', 'name', 'net_type', 'oneway',
       'ref', 'route_type', 'sequence', 'service', 'to', 'toll', 'tunnel',
       'unique_agency_id', 'unique_route_id', 'unique_trip_id', 'weight',
       'from_int', 'to_int'],
      dtype='object')

In [209]:
[x for x in set(edges['highway']) if x==x]

['residential',
 'pedestrian',
 'primary_link',
 'trunk_link',
 'tertiary',
 'primary',
 'secondary',
 'tertiary_link',
 'living_street',
 'secondary_link']

In [210]:
all_node_ids=set(list(edges['from_int'])+list(edges['to_int']))
LatLongDict={}
for node in all_node_ids:
    LatLongDict[node]=[nodes.loc[nodes['id_int']==node, 'y'].values[0], nodes.loc[nodes['id_int']==node, 'x'].values[0]]

In [211]:
LatLongDict[1670]

[42.365078, -83.130205]

In [212]:
network=pd.DataFrame()
network['aNodes']=edges['from_int']
network['bNodes']=edges['to_int']
network['aNodeLat']=network.apply(lambda row: LatLongDict[row['aNodes']][0], axis=1)
network['aNodeLon']=network.apply(lambda row: LatLongDict[row['aNodes']][1], axis=1)
network['bNodeLat']=network.apply(lambda row: LatLongDict[row['bNodes']][0], axis=1)
network['bNodeLon']=network.apply(lambda row: LatLongDict[row['bNodes']][1], axis=1)  
network['weight']=edges['weight']
network['distance']=edges['distance']
network['net_type']=edges['net_type']
    
network.head()

,aNodes,bNodes,aNodeLat,aNodeLon,bNodeLat,bNodeLon,weight,distance,net_type
0,1670,3017,42.365078,-83.130205,42.364285,-83.128295,0.416667,NaN,transit
1,3017,3019,42.364285,-83.128295,42.362877,-83.124738,0.750000,NaN,transit
2,3019,3023,42.362877,-83.124738,42.361449,-83.121102,0.800000,NaN,transit
3,3023,3020,42.361449,-83.121102,42.360092,-83.117825,0.700000,NaN,transit
4,3020,3024,42.360092,-83.117825,42.358725,-83.114490,0.800000,NaN,transit


In [213]:
%matplotlib inline

In [214]:
from ipyleaflet import Map, Polyline, basemaps, basemap_to_tiles, Circle, Marker

In [215]:
center= [network.iloc[0]['aNodeLat'], network.iloc[0]['aNodeLon']]

In [216]:
polylines=[]
m = Map(center = center, zoom =14, layers=[basemap_to_tiles(basemaps.CartoDB.DarkMatter)])
lines=[]
for ind, row in network.iterrows():
    lines.append([[float(row['aNodeLat']), float(row['aNodeLon'])], [float(row['bNodeLat']), float(row['bNodeLon'])]])

lineLayer = Polyline(
    locations = lines,
    weight=1,
    opacity=1,
    color='white',
    fill=False
)
m.add_layer(lineLayer)
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

# Find subset of links where at least one node is in the simulation area

In [217]:
print(len(LatLongDict))
sim_area_nodes=set()
# check if each node in any sim area, if so add to list
for node_num in LatLongDict:
    for feat in sim_area['features']:
        if shape(feat['geometry']).contains(Point(
            float(LatLongDict[node_num][1]), 
            float(LatLongDict[node_num][0]))):
            sim_area_nodes.add(node_num)
len(sim_area_nodes)

6744


1227

In [218]:
print(len(network))
network=network.loc[((network['aNodes'].isin(sim_area_nodes))|
    (network['bNodes'].isin(sim_area_nodes)))]
print(len(network))

36835
3245


In [219]:
polylines=[]
m = Map(center = center, zoom =15, layers=[basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)])
lines=[]
for ind, row in network.iterrows():
    lines.append([[float(row['aNodeLat']), float(row['aNodeLon'])], [float(row['bNodeLat']), float(row['bNodeLon'])]])

lineLayer = Polyline(
    locations = lines,
    weight=1,
    opacity=1,
    color='red',
    fill=False
)
m.add_layer(lineLayer)
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [220]:
# set(network['net_type'])

In [221]:
print(len(network))
copy_edges=network.copy()
for i in range(len(copy_edges)):
    temp_link=copy_edges.iloc[i].copy()
    temp_from_node = temp_link['aNodes']
    temp_from_node_lat = temp_link['aNodeLat']
    temp_from_node_lon = temp_link['aNodeLon']
    temp_link['aNodes']=temp_link['bNodes']
    temp_link['aNodeLat']=temp_link['bNodeLat']
    temp_link['aNodeLon']=temp_link['bNodeLon']
    temp_link['bNodes']=temp_from_node
    temp_link['bNodeLat']=temp_from_node_lat
    temp_link['bNodeLon']=temp_from_node_lon
    network.loc[len(network)]=temp_link.copy()

3245


In [222]:
print(len(network))

5235


In [145]:
import networkx as nx
G=nx.Graph()
for i, row in network.iterrows():
    G.add_edge(int(row['aNodes']), int(row['bNodes']), 
               weight=row['weight'])

In [146]:
len(G.edges)

1528

In [179]:
polylines=[]
m = Map(center = center, zoom =15, layers=[basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)])
lines=[]
for edge in G.edges:
    a_node, b_node=edge
    a_node_lon, a_node_lat=node_to_coordinates[a_node]
    b_node_lon, b_node_lat=node_to_coordinates[b_node]
    lines.append([[float(a_node_lat), float(a_node_lon)], [float(b_node_lat), float(b_node_lon)]])

lineLayer = Polyline(
    locations = lines,
    weight=1,
    opacity=1,
    color='red',
    fill=False
)
m.add_layer(lineLayer)
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [147]:
fw_result=nx.floyd_warshall_predecessor_and_distance(G, weight='weight')

# Test the results

In [223]:
nodes_to_link_attributes={}
for ind, row in network.iterrows():
    nodes_to_link_attributes['{}_{}'.format(row['aNodes'], row['bNodes'])]={
        'weight': row['weight'],
        'distance': row['distance'],
        'from_coord': [float(row['aNodeLon']), float(row['aNodeLat'])],
        'to_coord': [float(row['bNodeLon']), float(row['bNodeLat'])]
    }

In [149]:
def get_node_path_from_fw(all_pred, from_node, to_node):
    if from_node==to_node:
        return []
    pred=to_node
    path=[pred]
    while not pred==from_node:
        pred=all_pred[from_node][pred]
        path.insert(0,pred)
    return path
        
def get_path_coords_distances(drive_edges, path):
    coords, distances =[], []
    for node_ind in range(len(path)-1):
        from_node=path[node_ind]
        to_node=path[node_ind+1]
        link_attributes=nodes_to_link_attributes['{}_{}'.format(from_node, to_node)]
        distances+=[link_attributes['distance']]
        coords+=[link_attributes['from_coord']]
    coords+= [link_attributes['to_coord']]
    # add the final coordinate of the very last segment
    return coords, distances

In [224]:
coords, distances =[], []
for node_ind in range(len(path)-1):
    from_node=path[node_ind]
    print(from_node)
    to_node=path[node_ind+1]
    link_attributes=nodes_to_link_attributes['{}_{}'.format(from_node, to_node)]
    distances+=[link_attributes['distance']]
    coords+=[link_attributes['from_coord']]
    print(coords)
coords+= [link_attributes['to_coord']]
print(coords)

519
[[-83.069443, 42.328008]]
4161
[[-83.069443, 42.328008], [-83.065484, 42.329281]]
[[-83.069443, 42.328008], [-83.065484, 42.329281], [-83.06656, 42.330639]]


In [196]:
path=get_node_path_from_fw(fw_result[0], 519, 3367)

In [228]:
coords, distances=get_path_coords_distances(network, path)

In [229]:
coords

[[-83.069443, 42.328008], [-83.065484, 42.329281], [-83.06656, 42.330639]]

In [230]:
polylines=[]
m = Map(center = center, zoom =14, layers=[basemap_to_tiles(basemaps.CartoDB.DarkMatter)])
lines=[]
for ic in range(len(coords)-1):
    lines.append([[float(coords[ic][1]), float(coords[ic][0])], [float(coords[ic+1][1]), float(coords[ic+1][0])]])

lineLayer = Polyline(
    locations = lines,
    weight=1,
    opacity=1,
    color='white',
    fill=False
)
m.add_layer(lineLayer)
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [163]:
all_nodes=set(list(network['aNodes'])+list(network['bNodes']))
node_to_coordinates={}
for node in LatLongDict:
    if node in all_nodes:
        node_to_coordinates[node]=[float(LatLongDict[node][1]), float(LatLongDict[node][0])]

In [231]:
def get_activity_type(row):
    if row['net_type']=='transit':
        return 'pt'
    else:
        return 'walking'
network['activity']=network.apply(lambda row: get_activity_type(row), axis=1)

In [232]:
network['minutes']=network['weight']

In [ ]:
from transport_network import dict_to_gzip

In [235]:
# json.dump(fw_result[0], open(FLOYD_PREDECESSOR_PATH, 'w'))
dict_to_gzip(fw_result[0],'./cities/Detroit/clean/fw_result_pt.txt.gz')
network.to_csv(INT_NET_DF_FLOYD_PATH)